# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("cities.csv")

In [3]:
cities_df = cities_df[["City", "Country", "Lng","Lat","Max Temp","Humidity","Cloudiness","Wind Speed"]]
cities_df.head()

,City,Country,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed
0,dunedin,NZ,170.50,-45.87,281.48,83,0,1.34
1,ronne,DK,14.71,55.10,289.15,72,47,7.70
2,cape town,ZA,18.42,-33.93,289.26,87,0,0.50
3,punta arenas,CL,-70.92,-53.15,279.82,87,100,4.10
4,hilo,US,-155.09,19.73,302.15,62,40,2.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights= weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

In [5]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# I converted the suggested max temp bounds to kelvin
cities_df = cities_df.loc[cities_df["Max Temp"] < 299.817]
cities_df = cities_df.loc[cities_df["Max Temp"] > 294.261]
cities_df = cities_df.loc[cities_df["Cloudiness"] == 0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_df.reset_index()
hotel_df = hotel_df[["City", "Country", "Lng","Lat","Max Temp","Humidity","Cloudiness","Wind Speed"]]
hotel_df.head()

,City,Country,Lng,Lat,Max Temp,Humidity,Cloudiness,Wind Speed
0,albany,US,-73.97,42.60,298.71,53,0,5.52
1,maceio,BR,-35.74,-9.67,298.15,65,0,4.10
2,karratha,AU,116.85,-20.74,294.82,53,0,3.00
3,arraial do cabo,BR,-42.03,-22.97,299.15,83,0,4.60
4,sao joao da barra,BR,-41.05,-21.64,299.15,69,0,7.20


In [8]:
hotels = []

for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]['Lat']
    lng = hotel_df.loc[x]['Lng']
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params = {
        "location": f"{lat},{lng}", 
        "radius": 5000,
        "type": "hotel",
        "key": g_key,
    }
    
    response = requests.get(base_url, params=params).json()
    
    
    result = response["results"]
    
    if len(result) > 0:
        hotel_name = result[0]["name"]
        hotels.append(hotel_name)
    else:
        hotels.append("Error")
        
hotel_df["Hotel Name"] = hotels

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))